<a href="https://colab.research.google.com/github/gauravrana100/practice_transformers/blob/main/final_run_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os, json

In [3]:
os.chdir('/content/gdrive/MyDrive/Transformer-transfer-learning/MLM-keras')

# Prepare data

In [4]:
with open('f2f_dld_data', 'r') as f:
    data = f.read()

In [5]:
data_ls = [json.loads(el)['resp'] for el in data.split('------------')[:-1]]

In [6]:
def create_sentence(output):
    sentence = ''
    for vals in output:
        if vals['token'] in [',', '.']:
            sentence = sentence + vals['token']
        else:
            sentence = sentence + ' ' + vals['token']
    return sentence

In [7]:
final_d = list(map(lambda x: create_sentence(x).strip().lower(), data_ls))

In [8]:
final_d[:5]

['the human registers customer  with customer name, customer address, customer mobile number, customer email id, customer id, name, address, mobile no, salutation, customer full name, customer id proof, customer date of birth, customer language, customer gender, device location, delivery address, billing address, first_name, last_name, mobile_number, address1, address2, mandal, district, village_name, pincode, email and want_to_register.',
 'the human selects pizza from menu card with pizza type, pizza crust, pizza_size, pizza name, pizza combo id, pizza description, toppings, add ons and rate card.',
 'the human selects pizza from order_details with item price, item quantity, delivery charges, discount price, order id and order date_time.',
 'the human selects pizza via coupon with coupon id.',
 'the machine calculates pizza bill by customer with customer id, customer name, customer address and customer mobile number.']

In [9]:
! pip install datasets transformers

     |████████████████████████████████| 245kB 5.4MB/s 
     |████████████████████████████████| 2.5MB 10.0MB/s 
     |████████████████████████████████| 122kB 37.0MB/s 
     |████████████████████████████████| 245kB 35.0MB/s 
     |████████████████████████████████| 901kB 39.4MB/s 
     |████████████████████████████████| 3.3MB 34.6MB/s 
ERROR: transformers 4.7.0 has requirement huggingface-hub==0.0.8, but you'll have huggingface-hub 0.0.10 which is incompatible.


# Prepare MLM dataset

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame(columns=['text'], data=final_d)

In [13]:
df.head()

,text
0,the human registers customer with customer na...
1,the human selects pizza from menu card with pi...
2,the human selects pizza from order_details wit...
3,the human selects pizza via coupon with coupon...
4,the machine calculates pizza bill by customer ...


In [38]:
from sklearn.model_selection import train_test_split

In [39]:
train, test = train_test_split(df[['text']], test_size=0.05)

In [40]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [41]:
train_dataset

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 4550
})

#Modelling

In [16]:
from transformers import AutoTokenizer

In [22]:
model_checkpoint = "roberta-large"

In [18]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

block_size = 128

def group_texts(examples):
    # print('examples --', examples)
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    # print('result --', result)
    return result

In [42]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
train_tokenized_datasets = train_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text", "__index_level_0__"])
test_tokenized_datasets = test_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text", "__index_level_0__"])

In [43]:
train_lm_datasets = train_tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

test_lm_datasets = test_tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

First we use a model suitable for masked LM:

In [25]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [26]:
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNor

Freeze layers except last two

In [28]:
for name, params in model.named_parameters():
  print(name, params.shape, params.requires_grad)

roberta.embeddings.word_embeddings.weight torch.Size([50265, 1024]) True
roberta.embeddings.position_embeddings.weight torch.Size([514, 1024]) True
roberta.embeddings.token_type_embeddings.weight torch.Size([1, 1024]) True
roberta.embeddings.LayerNorm.weight torch.Size([1024]) True
roberta.embeddings.LayerNorm.bias torch.Size([1024]) True
roberta.encoder.layer.0.attention.self.query.weight torch.Size([1024, 1024]) True
roberta.encoder.layer.0.attention.self.query.bias torch.Size([1024]) True
roberta.encoder.layer.0.attention.self.key.weight torch.Size([1024, 1024]) True
roberta.encoder.layer.0.attention.self.key.bias torch.Size([1024]) True
roberta.encoder.layer.0.attention.self.value.weight torch.Size([1024, 1024]) True
roberta.encoder.layer.0.attention.self.value.bias torch.Size([1024]) True
roberta.encoder.layer.0.attention.output.dense.weight torch.Size([1024, 1024]) True
roberta.encoder.layer.0.attention.output.dense.bias torch.Size([1024]) True
roberta.encoder.layer.0.attention.o

In [30]:
for params in list(model.parameters())[:-5]:
  params.requires_grad = False

In [31]:
for name, params in model.named_parameters():
  print(name, params.shape, params.requires_grad)

roberta.embeddings.word_embeddings.weight torch.Size([50265, 1024]) False
roberta.embeddings.position_embeddings.weight torch.Size([514, 1024]) False
roberta.embeddings.token_type_embeddings.weight torch.Size([1, 1024]) False
roberta.embeddings.LayerNorm.weight torch.Size([1024]) False
roberta.embeddings.LayerNorm.bias torch.Size([1024]) False
roberta.encoder.layer.0.attention.self.query.weight torch.Size([1024, 1024]) False
roberta.encoder.layer.0.attention.self.query.bias torch.Size([1024]) False
roberta.encoder.layer.0.attention.self.key.weight torch.Size([1024, 1024]) False
roberta.encoder.layer.0.attention.self.key.bias torch.Size([1024]) False
roberta.encoder.layer.0.attention.self.value.weight torch.Size([1024, 1024]) False
roberta.encoder.layer.0.attention.self.value.bias torch.Size([1024]) False
roberta.encoder.layer.0.attention.output.dense.weight torch.Size([1024, 1024]) False
roberta.encoder.layer.0.attention.output.dense.bias torch.Size([1024]) False
roberta.encoder.layer.

 The data_collator is a function that is responsible of taking the samples and batching them in tensors. By doing this step inside the data_collator, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a DataCollatorForLanguageModeling. We can adjust the probability of the masking:

In [32]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [33]:
from transformers import Trainer, TrainingArguments

In [34]:
training_args = TrainingArguments(
    "test-mlm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [44]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_lm_datasets,
    eval_dataset=test_lm_datasets,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
